In [ ]:
import pandas as pd

Introduction About the Data :
The dataset The goal is to predict price of given diamond (Regression Analysis).

There are 10 independent variables (including id):

id : unique identifier of each diamond

carat : Carat (ct.) refers to the unique unit of weight measurement used exclusively to weigh gemstones and diamonds.

cut : Quality of Diamond Cut

color : Color of Diamond

clarity : Diamond clarity is a measure of the purity and rarity of the stone, graded by the visibility of these characteristics under 10-power magnification.

depth : The depth of diamond is its height (in millimeters) measured from the culet (bottom tip) to the table (flat, top surface)

table : A diamond's table is the facet which can be seen when the stone is viewed face up.

x : Diamond X dimension

y : Diamond Y dimension

x : Diamond Z dimension

Target variable:

price: Price of the given Diamond.

In [ ]:
import os
print(os.getcwd())
data=pd.read_csv("../data/train.csv")

In [ ]:
data.head(2)

In [ ]:
data["color"].value_counts()

In [ ]:
data.sample(10)

In [ ]:
data.isnull().sum()

In [ ]:
data.size

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.columns

In [ ]:
data.drop(labels=["id"],axis=1,inplace=True)

In [ ]:
data

In [ ]:
data.duplicated().sum()

In [ ]:
data.info()

In [ ]:
cat_columns=data.columns[data.dtypes=="object"]

In [ ]:
num_columns=data.columns[data.dtypes!="object"]

In [ ]:
data[num_columns].describe().T

In [ ]:
data[cat_columns]

In [ ]:
for col in cat_columns:
    print(data[col].value_counts())
    print(f"here is the total count:{len(data[col].value_counts())}")
    print("=======================")

In [ ]:
data[num_columns]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(6,4))

for col in num_columns:
    sns.histplot(data=data,x=col,kde=True)
    print("\n")
    plt.show()

In [ ]:
sns.heatmap(data[num_columns].corr(),annot=True)

In [ ]:
data[cat_columns]

In [ ]:
# it is a oridinal encoding
cut_map={"Fair":1,"Good":2,"Very Good":3,"Premium":4,"Ideal":5}
clarity_map = {"I1":1,"SI2":2 ,"SI1":3 ,"VS2":4 , "VS1":5 , "VVS2":6 , "VVS1":7 ,"IF":8}
color_map = {"D":1 ,"E":2 ,"F":3 , "G":4 ,"H":5 , "I":6, "J":7}

In [ ]:
data["cut"]=data["cut"].map(cut_map)
data["clarity"]=data["clarity"].map(clarity_map)
data["color"]=data["color"].map(color_map)

In [ ]:
data

In [ ]:
import pandas as pd
data=pd.read_csv("../data/train.csv")

In [ ]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
data.columns

In [ ]:
X=data.drop(labels=["id","price"],axis=1)
y=data["price"]

In [ ]:
X

In [ ]:
y

In [ ]:
X.info()

In [ ]:
X.select_dtypes(include="object")

In [ ]:
X.select_dtypes(exclude="object")

In [ ]:
cat_cols=X.select_dtypes(include="object").columns

num_cols=X.select_dtypes(exclude="object").columns

In [ ]:
cat_cols

In [ ]:
num_cols

In [ ]:
X.select_dtypes(include="object").head()

In [ ]:
X["clarity"].value_counts()

In [ ]:
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [ ]:
num_pipeline=Pipeline(
    steps=[
        ("imputer",SimpleImputer()),
        ("scaler",StandardScaler())
    ]
)

In [ ]:
cat_pipeline=Pipeline(

    steps=[
        ("imputer",SimpleImputer(strategy="most_frequent")),
        ("ordinalencoder",OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories]))

    ]
)

In [ ]:
num_cols

In [ ]:
cat_cols

In [ ]:
preprocessor=ColumnTransformer(

    [
        ("num_pipeline",num_pipeline,num_cols),
        ("cat_pipeline",cat_pipeline,cat_cols)
    ]
)

In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30, random_state=42)

In [ ]:
X_test.shape

In [ ]:
y_train.head()

In [ ]:
preprocessor.fit_transform(X_train)

In [ ]:
preprocessor.transform(X_test)

In [ ]:
preprocessor.get_feature_names_out()

In [ ]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [ ]:
X_train

In [ ]:
X_test

# model training

In [ ]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [ ]:
models={

    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet(),
    'Randomforest':RandomForestRegressor(),
    'xgboost':XGBRegressor()

}

In [ ]:
trained_model_list=[]
model_list=[]
r2_list=[]

In [ ]:
list(models)

In [ ]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    print(model)
    

In [ ]:
models.keys()

In [ ]:
list(models.values())[1]

In [ ]:
def evaluate_model(true,pred):
    r2=r2_score(true,pred)
    mae=mean_absolute_error(true,pred)
    mse=mean_squared_error(true,pred)

    return mae, mse,r2

In [ ]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    
    model.fit(X_train,y_train)

    #make_prediction
    y_pred=model.predict(X_test)

    #this is for the validaiton
    MAE,MSE,R2=evaluate_model(y_test,y_pred)


    print("model training performance",model)
    print("MSE:", MSE)
    print("MAE:",MAE)
    print("R2 SCORE:",R2)

    r2_list.append(R2)

    print("="*40)
    print("\n")

    

    

In [ ]:
r2_list

In [ ]:
max(r2_list)

In [ ]:
import logging
import os
from datetime import datetime

In [ ]:
LOG_FILE=f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

log_path=os.path.join(os.getcwd(),"logs")

os.makedirs(log_path,exist_ok=True)

LOG_FILEPATH=os.path.join(log_path,LOG_FILE)


In [ ]:
LOG_FILEPATH

In [ ]:
logging.basicConfig(level=logging.INFO, 
                    filename=LOG_FILEPATH,
                    format="[%(asctime)s] %(lineno)d %(name)s - %(levelname)s - %(message)s"
                    
)


In [ ]:
logging.info("this is my test log")